<a href="https://colab.research.google.com/github/wkd-woo/finance-statements/blob/main/%EC%9E%AC%EB%AC%B4%EC%9D%BC%EA%B4%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U finance-datareader

     |████████████████████████████████| 48 kB 2.5 MB/s 


In [2]:
import pandas as pd
import os
from tqdm import tqdm
import FinanceDataReader as fdr

# 파일별로 분류

In [3]:
import glob

재무상태표 = [] # _01_
손익계산서 = [] # _02_
포괄손익계산서 = [] # _03_
현금흐름표 = [] # _04_
자본변동표 = [] #_05_


for name in glob.glob('/content/drive/MyDrive/22캡스톤디자인/재무일괄/*.txt'):
  if '_01_' in name:
    재무상태표.append(name)
  elif '_02_' in name:
    손익계산서.append(name)
  elif '_03_' in name:
    포괄손익계산서.append(name)
  elif '_04_' in name:
    현금흐름표.append(name)
  elif '_05_' in name:
    자본변동표.append(name)

In [6]:
sample = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

# 데이터 추출

In [7]:
for path in tqdm(재무상태표):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기말' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기말' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기말' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[52:56] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # 자산총계(ifrs_Assets)
  자산총계 = df[(df['항목코드'] == 'ifrs_Assets') | (df['항목코드'] == 'ifrs-full_Assets') | (df['항목명'].isin(['자산 총계', '자산총계']))]
  자산총계.rename(columns={당기:"당기"}, inplace=True)
  자산총계.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(자산총계, sort=False, ignore_index=True)

  # 자본총계(ifrs_Equity)
  자본총계 = df[(df['항목코드'] == 'ifrs_Equity') | (df['항목코드'] == 'ifrs-full_Equity') | (df['항목명'].isin(['자본총계','자본 총계']))]
  자본총계.rename(columns={당기:"당기"}, inplace=True)
  자본총계.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(자본총계, sort=False, ignore_index=True)

  # 부채총계(ifrs_Liabilities)
  부채총계 = df[(df['항목코드'] == 'ifrs_Liabilities') | (df['항목코드'] == 'ifrs-full_Liabilities') | (df['항목명'].isin(['부채총계','부채 총계']))]
  부채총계.rename(columns={당기:"당기"}, inplace=True)
  부채총계.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(부채총계, sort=False, ignore_index=True)

  # 유동자산(ifrs_CurrentAssets)
  유동자산 = df[(df['항목코드'] == 'ifrs_CurrentAssets') | (df['항목코드'] == 'ifrs-full_CurrentAssets') | (df['항목명'].isin(['유동자산', '유동 자산']))]
  유동자산.rename(columns={당기:"당기"}, inplace=True)
  유동자산.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(유동자산, sort=False, ignore_index=True)

  # 현금 및 현금성자산(ifrs_CashAndCashEquivalents)
  현금성자산 = df[(df['항목코드'] == 'ifrs_CashAndCashEquivalents') | (df['항목코드'] == 'ifrs-full_CashAndCashEquivalents') 
  | df['항목명'].isin(['현금', '현금성자산', '현금 및 현금성자산', '현금및현금성자산']) | df['항목명'].str.contains('현금및현금성자산|현금 및 현금성자산')]
  현금성자산.rename(columns={당기:"당기"}, inplace=True)
  현금성자산.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(현금성자산, sort=False, ignore_index=True)


  0%|          | 0/24 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
 21%|██        | 5/24 [00:05<00:20,  1.08s/it]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 24/24 [00:28<00:00,  1.20s/it]


# 데이터 전처리

In [10]:
sample['종목코드'] = sample['종목코드'].apply(lambda x:x[1:-1])

In [14]:
sample['항목명'] = sample.apply(lambda x: '자산총계' if x['항목코드'] == 'ifrs_Assets' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '자산총계' if x['항목코드'] == 'ifrs-full_Assets' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '부채총계' if x['항목코드'] == 'ifrs_Liabilities' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '부채총계' if x['항목코드'] == 'ifrs-full_Liabilities' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '자본총계' if x['항목코드'] == 'ifrs_Equity' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '자본총계' if x['항목코드'] == 'ifrs-full_Equity' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '유동자산' if x['항목코드'] == 'ifrs_CurrentAssets' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '유동자산' if x['항목코드'] == 'ifrs-full_CurrentAssets' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '현금및현금성자산' if x['항목코드'] == 'ifrs_CashAndCashEquivalents' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '현금및현금성자산' if x['항목코드'] == 'ifrs-full_CashAndCashEquivalents' else x['항목명'], axis=1)

sample['항목명'] = sample.apply(lambda x: '자산총계' if x['항목명'] == '자산 총계' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '자본총계' if x['항목명'] == '자본 총계' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '부채총계' if x['항목명'] == '부채 총계' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '유동자산' if x['항목명'] == '유동 자산' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금성자산' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '현금및현금성자산' if x['항목명'] == '기타현금및현금성자산' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금 및 현금성자산' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금및현금성자산-증권업' else x['항목명'], axis=1)

sample

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,3분기보고서,3S,262,ifrs-full_Assets,자산총계,"65,694,406,823"
1,095570,2021,3분기보고서,AJ네트웍스,763,ifrs-full_Assets,자산총계,"1,357,720,740,483"
2,006840,2021,3분기보고서,AK홀딩스,649,ifrs-full_Assets,자산총계,"4,361,681,811,190"
3,054620,2021,3분기보고서,APS홀딩스,649,ifrs-full_Assets,자산총계,"287,247,655,273"
4,265520,2021,3분기보고서,AP시스템,292,ifrs-full_Assets,자산총계,"445,457,356,474"
...,...,...,...,...,...,...,...,...
192417,069260,2016,반기보고서,휴켐스,204,ifrs_CashAndCashEquivalents,현금및현금성자산,"38,326,395,822"
192418,010240,2016,반기보고서,흥국,292,ifrs_CashAndCashEquivalents,현금및현금성자산,"2,655,098,424"
192419,189980,2016,반기보고서,흥국에프엔비,112,ifrs_CashAndCashEquivalents,현금및현금성자산,"1,107,330,416"
192420,003280,2016,반기보고서,흥아해운,501,ifrs_CashAndCashEquivalents,현금및현금성자산,"20,664,686,060"


In [16]:
sample['항목명'].unique()

array(['자산총계', '자본총계', '부채총계', '유동자산', '현금및현금성자산'], dtype=object)

In [17]:
sample['항목코드'].unique()

array(['ifrs-full_Assets',
       'entity00138242_udf_BS_2018115102250529_StatementOfFinancialPositionAbstract',
       'ifrs-full_Equity',
       'entity00222532_udf_BS_2018425201848316_StatementOfFinancialPositionAbstract',
       'entity00545716_udf_BS_2021111611252554_StatementOfFinancialPositionAbstract',
       'entity00136271_udf_BS_20188817541842_StatementOfFinancialPositionAbstract',
       'entity00138242_udf_BS_2018115122023385_StatementOfFinancialPositionAbstract',
       'entity00188089_udf_BS_2017111113217301_StatementOfFinancialPositionAbstract',
       'entity00428251_udf_BS_20195814529840_StatementOfFinancialPositionAbstract',
       'ifrs-full_Liabilities',
       'entity00545716_udf_BS_2021111611286128_StatementOfFinancialPositionAbstract',
       'ifrs-full_CurrentAssets', 'ifrs-full_CashAndCashEquivalents',
       'ifrs-full_Cash', 'ifrs-full_CashEquivalents',
       'entity00602136_udf_BS_20211115143929453_CurrentAssets',
       'entity00117708_udf_BS_202031311591

In [18]:
재무상태표일괄 = sample

# 저장

In [19]:
재무상태표일괄.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/재무상태표일괄.csv')